In [1]:
import pandas as pd
import numpy as np
import os
import glob
from tasks import compute_index
from tasks.post_process import compare_2020_2019_data_report

# For the results

In [29]:
def make_2019_2020_correlation_report():
    
    data_2019 = pd.read_csv('data/2019_archive/result.csv').assign(version='v_2019')
    data_2020 = pd.read_csv('data/full_data/result.csv').assign(version='v_2020')
    data = pd.concat([data_2019, data_2020], axis=0).dropna(subset=['Value'])
    ISO_with_index = data.query("Aggregation == 'Index'").dropna().ISO.unique() # Select only ISOs where the full index is computed to remove some noise
    data = data[data.ISO.isin(ISO_with_index)]
    
    pivoted_data = data.pivot(index=['Variable', 'Aggregation', 'ISO', 'Year'], columns=['version'], values='Value')
    
    corr_by_var = pivoted_data.groupby(['Variable', 'Aggregation']).apply(lambda x: x[['v_2019', 'v_2020']].dropna().corr().values[0, 1]).to_frame(name='corr')
    corr_by_var_ISO = pivoted_data.groupby(['Variable', 'Aggregation', 'ISO']).apply(lambda x: x[['v_2019', 'v_2020']].dropna().corr().values[0, 1]).to_frame(name='corr')


    return pivoted_data, corr_by_var.reset_index(), corr_by_var_ISO.reset_index()


def make_2019_2020_correlation_report_bis():
    
    data_2019 = pd.read_csv('data/2019_archive/data.csv').assign(version='v_2019')
    data_2020 = pd.read_csv('data/full_data/data.csv').assign(version='v_2020')
    data = pd.concat([data_2019, data_2020], axis=0).dropna(subset=['Value'])
    
    pivoted_data = data.pivot(index=['Indicator', 'ISO', 'Year'], columns=['version'], values='Value')
    
    corr_by_var = pivoted_data.groupby(['Indicator']).apply(lambda x: x[['v_2019', 'v_2020']].dropna().corr().values[0, 1]).to_frame(name='corr')
    corr_by_var_ISO = pivoted_data.groupby(['Indicator', 'ISO']).apply(lambda x: x[['v_2019', 'v_2020']].dropna().corr().values[0, 1]).to_frame(name='corr')


    return pivoted_data, corr_by_var.reset_index(), corr_by_var_ISO.reset_index()

In [30]:
data, corr_by_var , corr_by_var_ISO = make_2019_2020_correlation_report()

In [31]:
corr_by_var.query('Aggregation == "Indicator" and corr < 0.9')

,Variable,Aggregation,corr
47,GE1,Indicator,0.797365
51,GE3,Indicator,0.897994
58,GN1,Indicator,0.209781
76,SE2,Indicator,0.460502


In [32]:
corr_by_var.query('Aggregation == "Indicator_normed" and corr < 0.9')

,Variable,Aggregation,corr
48,GE1,Indicator_normed,0.768254
59,GN1,Indicator_normed,0.584670
83,SL1,Indicator_normed,0.693383


In [35]:
corr_by_var.query('Aggregation == "Category" and corr < 0.9')

,Variable,Aggregation,corr
57,GN,Category,0.584670
67,ME,Category,0.650204
81,SL,Category,0.763839


In [36]:
corr_by_var.query("Variable == 'GN1'")

,Variable,Aggregation,corr
58,GN1,Indicator,0.209781
59,GN1,Indicator_normed,0.584670


# SL1:

Problem from last year's version corrected today (Large outliers that where not accounted for and that are now removed.)


New data includes less points some of the values are not computed (eg the outliers, HKG, SGP etc)

**Conclusion**: Using SL1 v_2020 seems more appropriate 

In [ ]:
data.loc['SL1', 'Indicator_normed', : ,2019]['v_2019'].sort_values().head(15)

# GE1

Values are different from last year despite multiple APIs check. It is unclear why the values are different.

TO CHECK but not clear what's going on

In [ ]:
corr_by_var_ISO.query('Variable == "GE1" and Aggregation == "Indicator"').sort_values(by='corr').head(10)

In [ ]:
pd.read_csv('data/indicator/GE1/preprocessed/GE1.0_CW.csv')#.query('ISO == "FRA"')

In [ ]:
pd.read_csv('data/indicator/GE1/computed/GE1.csv').query('ISO == "FRA"')

In [ ]:
data.loc['GE1', 'Indicator', 'FRA']

In [ ]:
data.loc['GE1', 'Indicator', 'ITA' , :]#['v_2019'].sort_values().head(15)

In [ ]:
config = {
            "GGI_code": "GE1.0",
            "params": {
              "gas_ids[]": 326,
              "sector_ids[]": [1299],
              "source_ids[]": [111]
            }
        }

In [ ]:
from download.downloaders import download, CW_Downloader
from processing.api_preprocessors import CW_Preprocessor

In [ ]:
test = download(API_name="CW", path=None, config=config)
test = CW_Preprocessor('GE1.0').json_to_pandas(test['data'])

In [ ]:
pop = pd.read_csv('data/indicator/GE1/processed/GE1.1_WB.csv')

In [ ]:
(test.query('iso_code3== "FRA"').set_index('year').value * 1e6 / pop.query('ISO == "FRA"').set_index('Year')['Value']).dropna()

In [ ]:
data.loc['GE1', 'Indicator', 'FRA']

In [ ]:
pd.read_csv('data/indicator/GE1/processed/GE1.0_CW.csv')

In [ ]:
def process_GE1_0(df):
    df = df.copy()
    piv = df.pivot(index=['iso_code3', 'year'], columns=['sector'], values='value')
    piv['value'] = (piv['Total excluding LUCF'] - piv['Agriculture']) * 1e6
    piv = (
        piv['value'].reset_index().assign(Description='Total excluding LUCF and Agriculture')
                    .assign(data_source='CAIT _and_ PIK')
    )
    
    
    return piv.query('value > 0')

In [ ]:
process_GE1_0(test)

# GE3 

Same as GE1, not clear what the problem is and where it comes from


In [ ]:
config = {
            "GGI_code": "GE3.1",
            "params": {
                "gas_ids[]": [
                    327,
                    328,
                    329
                ],
                "sector_ids[]":[
                    1304,
                    1302
                ],
                "source_ids[]": 111
            }
        }


test = download(API_name="CW", path=None, config=config)


In [ ]:
test = CW_Preprocessor('GE2.0').json_to_pandas(test['data'])

In [ ]:
test

In [ ]:
CW_Downloader('https://www.climatewatchdata.org/api/v1/data/historical_emissions').get_CW_API_ids("sectors")

#  ME1 

Check for a different source

In [ ]:
data.loc['ME1', 'Indicator', 'DEU']

# GN1 

Values are one the right order of magnitude but change from one version to the other, should double check properly and redownload.

In [14]:
data, corr_by_var , corr_by_var_ISO = make_2019_2020_correlation_report_bis()

In [16]:
corr_by_var.query('corr < 0.9')

,Indicator,corr
19,GE1,0.725332
21,GE3,0.865160
23,GN1,0.398926
26,ME1,0.833169
29,SE2,0.413501


In [25]:
corr_by_var_ISO.query("Indicator == 'GN1'").sort_values(by='corr').dropna().head(20)

,Indicator,ISO,corr
4571,GN1,ZWE,-0.899680
4570,GN1,ZMB,-0.839739
4500,GN1,ECU,-0.680925
4519,GN1,ITA,-0.532242
4499,GN1,DZA,-0.381463
4484,GN1,BIH,-0.361716
4565,GN1,URY,-0.353846
4556,GN1,SVK,-0.199912
4511,GN1,HND,-0.160110
4544,GN1,PAN,-0.049358


In [27]:
data.loc['GN1', 'ITA']

version,v_2019,v_2020
Year,,
1980,0.011587,0.012263
1981,0.011526,0.012256
1982,0.011724,0.012500
1983,0.011556,0.012260
1984,0.011527,0.012294
1985,0.011616,0.012387
1986,0.011527,0.012333
1987,0.011621,0.012322
1988,0.011219,0.012312


In [11]:
corr_by_var_ISO.query('Variable == "GN1" and Aggregation == "Indicator"').sort_values(by='corr').dropna().tail(20)

,Variable,Aggregation,ISO,corr
6968,GN1,Indicator,GRC,0.902724
6947,GN1,Indicator,BRA,0.902921
7019,GN1,Indicator,URY,0.907945
6998,GN1,Indicator,NOR,0.920885
6961,GN1,Indicator,EGY,0.939394
7001,GN1,Indicator,PER,0.941347
6995,GN1,Indicator,MYS,0.943689
7000,GN1,Indicator,PAN,0.944666
7008,GN1,Indicator,SLV,0.948170
6963,GN1,Indicator,EST,0.956031


# SE2

In [ ]:
corr_by_var_ISO.query('Variable == "SE2" and Aggregation == "Indicator"').sort_values(by='corr').head(30)

In [ ]:
data.loc['SE2', 'Indicator', 'RWA']